In [43]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
import numpy as np
from pickle import dump


df2020 = pd.read_csv('data/all_tournaments_with_stats_for_DNN2020.csv').dropna()

df2020.label = df2020.label.replace('joueur0',0)
df2020.label = df2020.label.replace('joueur1',1)

df2020.pointsDropping0 = df2020.pointsDropping0.apply(lambda x: x.replace(',','.'))
df2020.pointsDropping1 = df2020.pointsDropping1.apply(lambda x: x.replace(',','.'))

df = pd.read_csv('data/all_tournaments_with_stats_for_DNN.csv').dropna()

df = pd.concat([df, df2020], axis=0)

index_to_delete = []
for i, phase in enumerate(df.phase):
    try:
        int(phase)
    except:
        index_to_delete.append(i)
        
df = df.drop(index_to_delete)

df = df.drop(['index','year','duree', 'player0','player1','weight0','weight1','surface','inOut','startCareer0','startCareer1',
             'winnerRate', 'looserRate', 'winnerRateSurface','looserRateSurface','phase'],axis=1)


df.aces1 = (df.aces1/(df.serviceGamePlayed1*4)).astype('float64')
df.aces0 = df.aces0/(df.serviceGamePlayed0*4)
df.doubleFautes1 = df.doubleFautes1/(df.serviceGamePlayed1*3.5)
df.doubleFautes0 = df.doubleFautes0/(df.serviceGamePlayed0*3.5)

df.RateFace2Face0 = df.RateFace2Face0/100
df.RateFace2Face1 = df.RateFace2Face1/100




df = df.dropna()

df.info()

/Users/matrousseau/.local/share/virtualenvs/Pronoo-RxTA2Kor/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (37,65) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 48629 entries, 3 to 326
Data columns (total 72 columns):
typeTournament                 48629 non-null float64
label                          48629 non-null float64
aces0                          48629 non-null float64
age0                           48629 non-null float64
breakPointsConverted0          48629 non-null float64
breakPointsFaced0              48629 non-null float64
breakPointsOpportunities0      48629 non-null float64
breakPointsSaved0              48629 non-null float64
doubleFautes0                  48629 non-null float64
firstServe0                    48629 non-null float64
firstServePointsWon0           48629 non-null float64
firstServeReturnPointsWon0     48629 non-null float64
height0                        48629 non-null float64
nextBest0                      48629 non-null float64
points0                        48629 non-null float64
pointsDropping0                48629 non-null object
ranking0                      

In [62]:
scaler = MinMaxScaler()

print(df.columns)
scaler.fit(df)

df_scaled = scaler.transform(df)
dataframe = pd.DataFrame(df_scaled, columns=df.columns)


#save the scaler
dump(scaler, open('data/scaler.pkl', 'wb'))

Index(['typeTournament', 'label', 'aces0', 'age0', 'breakPointsConverted0',
       'breakPointsFaced0', 'breakPointsOpportunities0', 'breakPointsSaved0',
       'doubleFautes0', 'firstServe0', 'firstServePointsWon0',
       'firstServeReturnPointsWon0', 'height0', 'nextBest0', 'points0',
       'pointsDropping0', 'ranking0', 'returnGamesPlayed0', 'returnGamesWon0',
       'returnPointsWon0', 'secondServePointsWon0',
       'secondServeReturnPointsWon0', 'serviceGamePlayed0', 'serviceGamesWon0',
       'totalPointsWon0', 'totalServicePointsWon0', 'tournCell0', 'aces1',
       'age1', 'breakPointsConverted1', 'breakPointsFaced1',
       'breakPointsOpportunities1', 'breakPointsSaved1', 'doubleFautes1',
       'firstServe1', 'firstServePointsWon1', 'firstServeReturnPointsWon1',
       'height1', 'nextBest1', 'points1', 'pointsDropping1', 'ranking1',
       'returnGamesPlayed1', 'returnGamesWon1', 'returnPointsWon1',
       'secondServePointsWon1', 'secondServeReturnPointsWon1',
       'se

In [46]:
train, test = train_test_split(dataframe, test_size=0.1)
train, val = train_test_split(train, test_size=0.1)

print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

train.head()

39389 train examples
4377 validation examples
4863 test examples


,typeTournament,label,aces0,age0,breakPointsConverted0,breakPointsFaced0,breakPointsOpportunities0,breakPointsSaved0,doubleFautes0,firstServe0,...,Left-Handed.1,Right-Handed.1,RateFace2Face0,RateFace2FaceSurface0,RateFace2Face1,RateFace2FaceSurface1,Forme0,Forme1,WinMean0,WinMean1
44313,0.000000,0.0,0.129286,0.36,0.47,0.967005,0.677817,0.61,0.158910,0.581818,...,0.0,1.0,1.00,1.00,0.00,0.00,0.500000,0.500000,0.622407,0.622407
25285,0.428571,0.0,0.346437,0.52,0.42,0.500000,0.366197,0.53,0.154842,0.454545,...,0.0,1.0,0.50,1.00,0.50,0.00,0.333333,0.333333,0.829876,0.829876
31312,0.000000,0.0,0.414269,0.52,0.36,0.114213,0.058099,0.62,0.132278,0.400000,...,0.0,1.0,0.38,0.35,0.62,0.65,0.500000,0.666667,0.622407,0.709544
19974,0.428571,1.0,0.615450,0.40,0.38,0.474619,0.617958,0.67,0.089515,0.672727,...,0.0,1.0,0.40,1.00,0.60,0.00,0.333333,0.666667,0.829876,0.120332
24742,1.000000,1.0,0.447094,0.44,0.36,0.119289,0.038732,0.72,0.126230,0.454545,...,0.0,1.0,1.00,1.00,0.00,0.00,0.333333,0.333333,0.829876,0.829876


In [47]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe['label']
    dataframe = dataframe.drop(['label'], axis=1)
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), np.array(labels)))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

batch_size = 64 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [48]:
for feature_batch, label_batch in train_ds.take(1):
  print('A batch of aces:', feature_batch['aces0'])
  print('A batch of targets:', label_batch )

A batch of aces: tf.Tensor(
[0.37904468 0.38778409 0.38516241 0.12272727 0.26888604 0.17922078
 0.15584416 0.19176136 0.07087112 0.20221729 0.26820693 0.21158201
 0.24114833 0.14354067 0.09559513 0.0855615  0.44919786 0.16231519
 0.1143695  0.23753666 0.32657788 0.34394904 0.49786455 0.35294118
 0.54545455 0.12301741 0.25494071 0.10909091 0.16975881 0.59916451
 0.27272727 0.22165114 0.33766234 0.16570771 0.43913713 0.41069519
 0.28156293 0.18759019 0.24242424 0.3853211  0.13636364 0.2775974
 0.13533835 0.24451411 0.30355731 0.09848485 0.12121212 0.26243568
 0.27702896 0.33075435 0.13963636 0.03896104 0.23420647 0.10026738
 0.22966507 0.33       0.21818182 0.18427518 0.38407746 0.29752066
 0.3969642  0.22038567 0.15508021 0.34710744], shape=(64,), dtype=float64)
A batch of targets: tf.Tensor(
[1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0.
 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1.
 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 0. 1.], 

In [49]:
example_batch = next(iter(train_ds))

In [50]:
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

In [51]:
feature_columns = []

for header in train.columns:
    if header!= 'label':
      feature_columns.append(feature_column.numeric_column(header))
    
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [59]:
import datetime

batch_size = 64
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)


# Define a simple sequential model
def create_model():
        
    model = tf.keras.Sequential([
        feature_layer,
        layers.Dense(1000, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(3000, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(3000, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(3000, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(3000, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(3000, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(3000, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

    opt = tf.keras.optimizers.SGD(lr=0.005, decay=1e-5, momentum=0.9, nesterov=True)
    model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])
    
    return model



model = create_model()

In [60]:
import os 
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)


model.fit(train_ds,
          validation_data=val_ds,
          epochs=30,
          callbacks=[tensorboard_callback, early_stopping])

model.save_weights('./checkpoint/my_checkpoint')

Train for 616 steps, validate for 69 steps
Epoch 1/30
616/616 [==============================] - 120s 194ms/step - loss: 0.6948 - accuracy: 0.5107 - val_loss: 0.6865 - val_accuracy: 0.6662
Epoch 2/30
616/616 [==============================] - 120s 194ms/step - loss: 0.6741 - accuracy: 0.5684 - val_loss: 0.5866 - val_accuracy: 0.6934
Epoch 3/30
616/616 [==============================] - 118s 191ms/step - loss: 0.6018 - accuracy: 0.6755 - val_loss: 0.5700 - val_accuracy: 0.7032
Epoch 4/30
616/616 [==============================] - 116s 189ms/step - loss: 0.5874 - accuracy: 0.6887 - val_loss: 0.5720 - val_accuracy: 0.7025
Epoch 5/30
616/616 [==============================] - 118s 192ms/step - loss: 0.5809 - accuracy: 0.6918 - val_loss: 0.5616 - val_accuracy: 0.7066
Epoch 6/30
616/616 [==============================] - 118s 191ms/step - loss: 0.5762 - accuracy: 0.6945 - val_loss: 0.5614 - val_accuracy: 0.7057
Epoch 7/30
616/616 [==============================] - 117s 191ms/step - loss: 0.5

In [61]:
model = create_model()

model.load_weights('./checkpoint/my_checkpoint')

loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

76/76 [==============================] - 2s 24ms/step - loss: 0.5592 - accuracy: 0.7164
Accuracy 0.7164302
